# Are “custom loss functions” simple, or super hard?

Huh. I got stuck trying to use a “custom loss function in my `Find_Conspicuous_Disk` model. I had been lead to expect it to be pretty simple (as discussed here:[in the Keras doc](https://keras.io/api/losses/#creating-custom-losses) and [this post](https://towardsdatascience.com/how-to-create-a-custom-loss-function-keras-3a89156ec69b)). But I kept running into problems (as described in this [SO question](https://stackoverflow.com/questions/62393032/custom-loss-function-with-weights-in-keras)). 

(Worth noting that I actually was trying first to add a new **metric** so I could see the output of my new loss function while still learning with the original loss function MSE.)

So I decided the Keras doc was out of date, showing the old simple way not the complex new way. So I made a small test case in this notebook. **And it worked**. I figured something must be wrong, it can't be that easy. Probably it is just ignoring my loss function. So I modified it (multiplying `squared_difference` (below) by 1000) and indeed it still worked, but the reported loss values were just 1000 times bigger.

**Tracking loss history over 100 epochs:**
*   For size 10, the loss goes from **0.6554** down to **0.1070**
*   For size 1000, the loss goes from **0.5931** down to **0.0785**



In [1]:
import random
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential

In [2]:
# Check for Colab Pro resources
def check_colab_resources():
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    if ram_gb < 20:
        print('Not using a high-RAM runtime.')
    else:
        print('Using a high-RAM runtime.')
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

check_colab_resources()

Mon Dec 13 02:09:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   27C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Reset random sequence seeds in Python's "random", Numpy, and TensorFlow.
random_seed = 24304832
def reset_random_seeds():
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

# Verson 1 — December 8, 2021

In [4]:
# Test custom loss function
def test(training_set_size=10):
    reset_random_seeds()

    # Original version of my_loss_fn()
    def my_loss_fn(y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(squared_difference, axis=-1)  # Note `axis=-1`

    # # A different version of my_loss_fn() -- which makes the loss 1000 times
    # # bigger -- to verify that this function is actually being used.
    # def my_loss_fn(y_true, y_pred):
    #     squared_difference = tf.square(y_true - y_pred) * 1000
    #     return tf.reduce_mean(squared_difference, axis=-1)  # Note `axis=-1`

    # Random training data of size train_set_count:
    #     training example, each as a pairs of float features on [0, 1]
    #     labels, each as a float on [0, 1]
    X_train = np.random.random_sample((training_set_size, 2))
    y_train = np.random.random_sample((training_set_size, 1))

    print(X_train.shape)
    print(y_train.shape)

    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(2,)))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=my_loss_fn)
    # model.fit(X_train, y_train, epochs=100)
    model.fit(X_train, y_train, epochs=100, batch_size=32)

test(10)
# test(1000)

(10, 2)
(10, 1)
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.6554
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6412
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6273
Epoch 4/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6135
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 0.5999
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.5867
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.5737
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5609
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5484
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5361
Epoch 11/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5240
Epoch 12/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5121
Epoch 13/100
1/1 [============================

# Version 2 — December 12, 2021

In [5]:
# epochs = 10
# default_tss = 10
epochs = 10
default_tss = 1000


# Construct/run simple model A with given loss.
def run_model_a(loss = 'mse', training_set_size = default_tss):
    X_train = np.random.random_sample((training_set_size, 2))
    y_train = np.random.random_sample((training_set_size, 1))
    print(X_train.shape)
    print(y_train.shape)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(2,)))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=loss)
    model.fit(X_train, y_train, epochs=epochs)

# Construct/run "complex" model A with given loss.
def run_model_b(loss = 'mse', training_set_size = default_tss):
    # X_train = np.random.random_sample((training_set_size, 2))
    s = 10
    X_train = np.random.random_sample((training_set_size, s, s, 1))
    y_train = np.random.random_sample((training_set_size, 1))
    print(X_train.shape)
    print(y_train.shape)
    model = Sequential()

    # model.add(Conv2D(4, (3, 3), activation='relu', input_shape=(s, s, 1)))
    model.add(Conv2D(4, (3, 3), activation='relu', padding='same', input_shape=(s, s, 1)))
    model.add(Conv2D(4, (3, 3), activation='relu', padding='same', input_shape=(s, s, 1)))
    
    
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=loss)
    model.fit(X_train, y_train, epochs=epochs)

def simple_loss_function(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

run_model_a()
run_model_b()
run_model_a(simple_loss_function)
run_model_b(simple_loss_function)

(1000, 2)
(1000, 1)
Epoch 1/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2476
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1622
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1137
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0955
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0917
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0907
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0892
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0879
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0868
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0859
(1000, 10, 10, 1)
(1000, 1)
Epoch 1/10
32/32 [==============================] - 7s 4ms/step - loss: 0.1243
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0946
Epo